# Berlin Data Preprocessing

Time series for the Havel River inflow to the city (Konradshöhe, Messstellennummer 305) and the downstream station (Schleuse Spandau, Messstellennummer 325), DOC and TOC.

In [ ]:
import os
import pandas as pd
import numpy as np

import random

import plotly.express as px
import plotly.graph_objects as go

import statsmodels.formula.api as smf

from sklearn.preprocessing import MinMaxScaler

from plotly.subplots import make_subplots
from prophet import Prophet
from sklearn.metrics import (
    mean_absolute_error,
    median_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)

from scipy import stats

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
data_folder = os.path.join("..", "data", "berlin")
raw_data_folder = os.path.join(data_folder, "raw")
clean_data_folder = os.path.join(data_folder, "clean")

## Load Data

In [ ]:
ts_sw_df = pd.read_csv(
    os.path.join(raw_data_folder, "time-series_surface-water_quality.csv")
)

In [ ]:
flow_df = pd.read_csv(
    os.path.join(raw_data_folder, "time-series_surface-water_flow.csv")
)

In [ ]:
meteo_df = pd.read_csv(
    os.path.join(
        raw_data_folder,
        "produkt_klima_tag_19480101_20231231_00433.csv",
    ),
    sep=";",
)

In [ ]:
ts_sw_df.rename(
    columns={
        "Messstelle": "Station",
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Vorzeichen": "Sign",
        "Wert": "Value",
        "Bestimmungsgrenze": "LOQ",
    },
    inplace=True,
)

ts_sw_df.drop(
    columns=[
        "Entnahmetiefe [m]",
        "Messmethode",
    ],
    inplace=True,
)

In [ ]:
flow_df.rename(
    columns={
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Tagesmittelwert": "Flow River",
    },
    inplace=True,
)

In [ ]:
meteo_df.rename(
    columns={
        "STATIONS_ID": "Station ID",
        "MESS_DATUM": "DateTime",
        "  FX": "Wind Speed Max (m/s)",
        "  FM": "Wind Speed Mean (m/s)",
        " RSK": "Cumulated Rainfall (mm)",
        "RSKF": "Cumulated Rainfall Type",
        " SDK": "Sunshine Duration (hours)",
        "SHK_TAG": "Snow Height (cm)",
        "  NM": "Cloud Coverage (1/8)",
        " VPM": "Vapor Pressure (hPa)",
        "  PM": "Pressure (hPa)",
        " TMK": "Temperature Mean (°C)",
        " UPM": "Humidity (%)",
        " TXK": "Temperature Max at 2m (°C)",
        " TNK": "Temperature Min at 2m (°C)",
        " TGK": "Temperature Min at 5cm (°C)",
    },
    inplace=True,
)

## Build Dataset per Station

In [ ]:
# the parameters that are present for the moment are:
variables = {
    "Lufttemperatur": "Air Temperature (°C)",
    "Wassertemperatur": "Water Temperature (°C)",
    "Spektraler Absorptionskoeffizient (SAK) 254nm": "UVA254 (1/m)",
    "Leitfähigkeit": "Conductivity (µS/cm)",
    "Ammonium-Stickstoff": "Ammonium (mg/l)",
    "Sauerstoff-Gehalt": "Dissolved Oxygen (mg/l)",
    "Nitrat-Stickstoff": "Nitrate (mg/l)",
    "pH-Wert": "pH",
    "DOC (Gelöster organischer Kohlenstoff)": "DOC (mg/l)",
    "TOC (Organischer Kohlenstoff)": "TOC (mg/l)",
    "Sulfat": "Sulphate (mg/l)",
}

In [ ]:
surface_df = ts_sw_df[ts_sw_df["Parameter"].isin(variables.keys())]

surface_df["Parameter"] = surface_df["Parameter"].map(variables)

In [ ]:
surface_df["DateTime"] = pd.to_datetime(surface_df["DateTime"])

In [ ]:
# get rows where Sign is not nan
surface_df[surface_df["Sign"].notna()]

In [ ]:
stations_dict = {}
for station in surface_df["Station ID"].unique().tolist():
    station_df = surface_df[surface_df["Station ID"] == station]
    station_df = station_df.pivot_table(
        index=pd.Grouper("DateTime"),
        columns="Parameter",
        values="Value",
    )

    stations_dict[station] = station_df

stations_dict.pop(105)

### Plot Variables

In [ ]:
# get common columns for all the stations
common_columns = set(stations_dict[305].columns)
for station_id, station_df in stations_dict.items():
    common_columns = common_columns.intersection(station_df.columns)

In [ ]:
for column in common_columns:
    fig = go.Figure()

    for station_id, station_df in stations_dict.items():
        column_df = station_df[column].copy()

        column_df.dropna(inplace=True)

        fig.add_trace(
            go.Scatter(
                x=column_df.index,
                y=column_df,
                mode="lines",
                name=f"Station {station_id}",
            )
        )

    fig.update_layout(
        title=column,
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
    )
    fig.show()

In [ ]:
surface_info_df = pd.DataFrame(
    index=pd.Index(
        [
            "N Samples",
            "% Missing Values",
            "Frequency (days)",
            "Mean",
            "Std",
            "Start Date",
            "End Date",
        ],
        name="Info",
    ),
    columns=pd.MultiIndex.from_product(
        [surface_df["Station ID"].unique(), variables.values()],
        names=["Station ID", "Parameter"],
    ),
)

### 305 - Oberhavel-Konradshöhe

In [ ]:
station_df = stations_dict[305]

In [ ]:
flow_df["DateTime"] = pd.to_datetime(flow_df["DateTime"])

station_flow_df = flow_df[flow_df["Station ID"] == 5815911]

station_flow_df = station_flow_df[["DateTime", "Flow River"]].set_index("DateTime")

station_flow_df.index = station_flow_df.index.date
station_df.index = station_df.index.date

# merge the flow data with the surface water data for the same date (just date, not time)
station_df = station_df.merge(
    station_flow_df, left_index=True, right_index=True, how="left"
)

station_df.rename(columns={"Flow River": "Flow River Rate (m³/s)"}, inplace=True)

station_df.index = pd.to_datetime(station_df.index)

In [ ]:
# for each column, compute the % of missing values
for column in station_df.columns:
    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    df = station_df[date_range[0] : date_range[1]][column]

    missing_values = df.isna().sum() / df.shape[0]
    print(f"{column}: {missing_values}")

#### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    # compute date range for which the data is available
    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 305 - Range: {date_range[0].date()} - {date_range[1].date()}",
        labels={"DateTime": "DateTime", column: column},
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
    )

    fig.show()

#### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    fig = go.Figure()
    column_df = station_df[column]

    for year in column_df.index.year.unique():
        fig.add_trace(go.Box(y=column_df[column_df.index.year == year], name=year))
    fig.update_layout(
        title=f"{column} at station 305",
        xaxis_title="Year",
        yaxis_title=column,
    )

    fig.show()

#### Invalid Values

In [ ]:
station_df.loc[
    (station_df["DOC (mg/l)"] <= 0) | (station_df["DOC (mg/l)"] >= 20),
    ["DOC (mg/l)"],
] = np.nan
station_df.loc[station_df["TOC (mg/l)"] <= 0, ["TOC (mg/l)"]] = np.nan
station_df.loc[station_df["Flow River Rate (m³/s)"] < 0, ["Flow River Rate (m³/s)"]] = (
    np.nan
)
station_df.loc[station_df["Ammonium (mg/l)"] < 0, ["Ammonium (mg/l)"]] = np.nan
station_df.loc[station_df["Nitrate (mg/l)"] < 0, ["Nitrate (mg/l)"]] = np.nan

In [ ]:
# boxplot of the data
for column in station_df.columns:
    fig = go.Figure()
    column_df = station_df[column]

    for year in column_df.index.year.unique():
        fig.add_trace(go.Box(y=column_df[column_df.index.year == year], name=year))
    fig.update_layout(
        title=f"{column} at station 305",
        xaxis_title="Year",
        yaxis_title=column,
    )

    fig.show()

#### Store Info

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    df = station_df[column].copy()

    start_date = df.dropna().index.min().strftime("%Y-%m-%d")
    end_date = df.dropna().index.max().strftime("%Y-%m-%d")

    df = df[start_date:end_date]

    missing_values = df.isna().sum() / df.shape[0] * 100

    surface_info_df.loc["N Samples", (305, column)] = (
        station_df[column].dropna().shape[0]
    )
    surface_info_df.loc["% Missing Values", (305, column)] = missing_values
    surface_info_df.loc["Frequency (days)", (305, column)] = (
        station_df.index.to_series().diff().value_counts().index[0].days
    )

    surface_info_df.loc["Mean", (305, column)] = df.mean()
    surface_info_df.loc["Std", (305, column)] = df.std()

    surface_info_df.loc["Start Date", (305, column)] = start_date
    surface_info_df.loc["End Date", (305, column)] = end_date

#### Outliers and Missing Values Imputation

In [ ]:
# for each column, compute the % of missing values
for column in station_df.columns:
    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    df = station_df[date_range[0] : date_range[1]][column]

    missing_values = df.isna().sum() / df.shape[0]
    print(f"{column}: {missing_values}")
    print()

In [ ]:
factor = 1.75

In [ ]:
# define the outliers through the STL decomposition

for column in station_df.columns:
    df = station_df[column].copy()

    df.dropna(inplace=True)

    df = df.resample("ME").mean()

    df.interpolate(method="time", inplace=True)

    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    # make sure that the dataframe starts and finishes in the same month
    start_index = df[df.index.month == date_range[1].month].index[0]

    # Slice the dataframe to start from the found index
    df = df.loc[start_index:]

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df,
            mode="lines",
            name="Original",
        ),
        row=1,
        col=1,
    )

    # ===== Prophet =====

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = forecasting_final["y"] - forecasting_final["yhat"]
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"] - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes" if (np.abs(x["error"]) > factor * x["uncertainty"]) else "No",
        axis=1,
    )

    print("===== Prophet =====")

    # Mean Absolute Error (MAE)
    MAE = mean_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Absolute Error (MAE): " + str(np.round(MAE, 2)))

    # Median Absolute Error (MedAE)
    MEDAE = median_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Median Absolute Error (MedAE): " + str(np.round(MEDAE, 2)))

    # Mean Squared Error (MSE)
    MSE = mean_squared_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Squared Error (MSE): " + str(np.round(MSE, 2)))

    # Root Mean Squarred Error (RMSE)
    RMSE = np.sqrt(
        int(mean_squared_error(forecasting_final["yhat"], forecasting_final["y"]))
    )
    print("Root Mean Squared Error (RMSE): " + str(np.round(RMSE, 2)))

    # Mean Absolute Percentage Error (MAPE)
    MAPE = mean_absolute_percentage_error(
        forecasting_final["yhat"], forecasting_final["y"]
    )
    print("Mean Absolute Percentage Error (MAPE): " + str(np.round(MAPE, 2)) + " %")

    anomaly = forecasting_final[forecasting_final["anomaly"] == "Yes"]

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["yhat"],
            mode="lines",
            name="Prediction (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["y"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["error"],
            mode="lines",
            name="Error",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["uncertainty"],
            mode="lines",
            name="Uncertainty",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["error"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=2,
        col=1,
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        height=800,
        width=1000,
    )

    fig.show()

In [ ]:
# Prophet is used to remove outliers

# create copy such that the processed columns do not affect the original dataframe until the end
station_df_copy = station_df.copy()

station_df_copy = station_df_copy.resample("M").mean()

for column in station_df.columns:
    df = station_df[column].copy()

    df.dropna(inplace=True)

    df = df.resample("M").mean()

    df.interpolate(method="time", inplace=True)

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = forecasting_final["y"] - forecasting_final["yhat"]
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"] - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes" if (np.abs(x["error"]) > factor * x["uncertainty"]) else "No",
        axis=1,
    )

    # remove the outliers
    forecasting_final = forecasting_final[forecasting_final["anomaly"] == "No"]

    df = forecasting_final[["ds", "y"]]

    df.set_index("ds", inplace=True)

    df.rename(columns={"y": column}, inplace=True)

    # redo the resampling since the outliers have been removed and
    # some months may have been removed
    df = df.resample("ME").mean()

    df.interpolate(method="time", inplace=True)

    station_df_copy.loc[df.index, column] = df[column]


station_df = station_df_copy

In [ ]:
# truncate the dataset based on the DOC date range
start_date = station_df["DOC (mg/l)"].dropna().index.min()
end_date = station_df["DOC (mg/l)"].dropna().index.max()

station_df = station_df[start_date:end_date]

In [ ]:
sw_305_df = station_df

### 325 - Havel-Pichelsdorfer Gemünd

In [ ]:
station_df = stations_dict[325]

In [ ]:
flow_df["DateTime"] = pd.to_datetime(flow_df["DateTime"])

station_flow_df = flow_df[flow_df["Station ID"] == 5803200]

station_flow_df = station_flow_df[["DateTime", "Flow River"]].set_index("DateTime")

station_flow_df.index = station_flow_df.index.date
station_df.index = station_df.index.date

# merge the flow data with the surface water data for the same date (just date, not time)
station_df = station_df.merge(
    station_flow_df, left_index=True, right_index=True, how="left"
)

station_df.rename(columns={"Flow River": "Flow River Rate (m³/s)"}, inplace=True)

station_df.index = pd.to_datetime(station_df.index)

#### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 325",
        labels={"DateTime": "DateTime", column: column},
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
    )
    fig.show()

#### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    fig = go.Figure()
    column_df = station_df[column]

    for year in column_df.index.year.unique():
        fig.add_trace(go.Box(y=column_df[column_df.index.year == year], name=year))
    fig.update_layout(
        title=f"{column} at station 325",
        xaxis_title="Year",
        yaxis_title=column,
    )

    fig.show()

#### Invalid Values

In [ ]:
station_df.loc[
    (station_df["DOC (mg/l)"] > 15) | (station_df["DOC (mg/l)"] < 4.5),
    ["DOC (mg/l)"],
] = np.nan
station_df.loc[station_df["TOC (mg/l)"] <= 0, ["TOC (mg/l)"]] = np.nan
station_df.loc[station_df["Flow River Rate (m³/s)"] < 0, ["Flow River Rate (m³/s)"]] = (
    np.nan
)
station_df.loc[station_df["Ammonium (mg/l)"] < 0, ["Ammonium (mg/l)"]] = np.nan
station_df.loc[station_df["Nitrate (mg/l)"] < 0, ["Nitrate (mg/l)"]] = np.nan
station_df.loc[station_df["pH"] < 7, ["pH"]] = np.nan

In [ ]:
# boxplot of the data
for column in station_df.columns:
    fig = go.Figure()
    column_df = station_df[column]

    for year in column_df.index.year.unique():
        fig.add_trace(go.Box(y=column_df[column_df.index.year == year], name=year))
    fig.update_layout(
        title=f"{column} at station 105",
        xaxis_title="Year",
        yaxis_title=column,
    )

    fig.show()

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    df = station_df[column].copy()

    start_date = df.dropna().index.min().strftime("%Y-%m-%d")
    end_date = df.dropna().index.max().strftime("%Y-%m-%d")

    df = df[start_date:end_date]

    missing_values = df.isna().sum() / df.shape[0] * 100

    surface_info_df.loc["N Samples", (325, column)] = (
        station_df[column].dropna().shape[0]
    )
    surface_info_df.loc["% Missing Values", (325, column)] = missing_values
    surface_info_df.loc["Frequency (days)", (325, column)] = (
        station_df.index.to_series().diff().value_counts().index[0].days
    )

    surface_info_df.loc["Mean", (325, column)] = df.mean()
    surface_info_df.loc["Std", (325, column)] = df.std()

    surface_info_df.loc["Start Date", (325, column)] = start_date
    surface_info_df.loc["End Date", (325, column)] = end_date

#### Outliers and Missing Values Imputation

In [ ]:
# for each column, compute the % of missing values
for column in station_df.columns:
    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    df = station_df[date_range[0] : date_range[1]][column]

    missing_values = df.isna().sum() / df.shape[0]
    print(f"{column}: {missing_values}")
    print()

In [ ]:
# define the outliers through Prophet

for column in station_df.columns:
    df = station_df[column].copy()

    df.dropna(inplace=True)

    df = df.resample("ME").mean()

    df.interpolate(method="time", inplace=True)

    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()

    # make sure that the dataframe starts and finishes in the same month
    start_index = df[df.index.month == date_range[1].month].index[0]

    # Slice the dataframe to start from the found index
    df = df.loc[start_index:]

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df,
            mode="lines",
            name="Original",
        ),
        row=1,
        col=1,
    )

    # ===== Prophet =====

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = forecasting_final["y"] - forecasting_final["yhat"]
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"] - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes" if (np.abs(x["error"]) > factor * x["uncertainty"]) else "No",
        axis=1,
    )

    print("===== Prophet =====")

    # Mean Absolute Error (MAE)
    MAE = mean_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Absolute Error (MAE): " + str(np.round(MAE, 2)))

    # Median Absolute Error (MedAE)
    MEDAE = median_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Median Absolute Error (MedAE): " + str(np.round(MEDAE, 2)))

    # Mean Squared Error (MSE)
    MSE = mean_squared_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Squared Error (MSE): " + str(np.round(MSE, 2)))

    # Root Mean Squarred Error (RMSE)
    RMSE = np.sqrt(
        int(mean_squared_error(forecasting_final["yhat"], forecasting_final["y"]))
    )
    print("Root Mean Squared Error (RMSE): " + str(np.round(RMSE, 2)))

    # Mean Absolute Percentage Error (MAPE)
    MAPE = mean_absolute_percentage_error(
        forecasting_final["yhat"], forecasting_final["y"]
    )
    print("Mean Absolute Percentage Error (MAPE): " + str(np.round(MAPE, 2)) + " %")

    anomaly = forecasting_final[forecasting_final["anomaly"] == "Yes"]

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["yhat"],
            mode="lines",
            name="Prediction (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["y"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["error"],
            mode="lines",
            name="Error",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["uncertainty"],
            mode="lines",
            name="Uncertainty",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["error"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=2,
        col=1,
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        height=800,
        width=1000,
    )

    fig.show()

In [ ]:
# Prophet is used to remove outliers

# create copy such that the processed columns do not affect the original dataframe until the end
station_df_copy = station_df.copy()

station_df_copy = station_df_copy.resample("M").mean()

for column in station_df.columns:
    df = station_df[column].copy()

    df.dropna(inplace=True)

    df = df.resample("M").mean()

    df.interpolate(method="time", inplace=True)

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = forecasting_final["y"] - forecasting_final["yhat"]
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"] - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes" if (np.abs(x["error"]) > factor * x["uncertainty"]) else "No",
        axis=1,
    )

    # remove the outliers
    forecasting_final = forecasting_final[forecasting_final["anomaly"] == "No"]

    df = forecasting_final[["ds", "y"]]

    df.set_index("ds", inplace=True)

    df.rename(columns={"y": column}, inplace=True)

    # redo the resampling since the outliers have been removed and
    # some months may have been removed
    df = df.resample("M").mean()

    df.interpolate(method="time", inplace=True)

    station_df_copy.loc[df.index, column] = df[column]


station_df = station_df_copy

In [ ]:
# truncate the dataset based on the DOC date range
start_date = station_df["DOC (mg/l)"].dropna().index.min()
end_date = station_df["DOC (mg/l)"].dropna().index.max()

station_df = station_df[start_date:end_date]

In [ ]:
sw_325_df = station_df

## Build Unique Dataset

In [ ]:
# build unique dataframe for all stations
# set the number of the station as further variable
sw_305_df["Station"] = 305
sw_325_df["Station"] = 325

sw_305_df.index.name = "DateTime"
sw_325_df.index.name = "DateTime"

sw_305_df.reset_index(inplace=True)
sw_325_df.reset_index(inplace=True)

# merge the dataframes
sw_df = pd.concat([sw_305_df, sw_325_df])

In [ ]:
# drop the TOC and BOD columns
sw_df.drop(columns=["TOC (mg/l)"], inplace=True)

### Meteorological

In [ ]:
diff_columns = ["QN_3", "QN_4", "eor", "Cumulated Rainfall Type"]

In [ ]:
meteo_df["DateTime"] = pd.to_datetime(meteo_df["DateTime"], format="%Y%m%d")

meteo_df.set_index("DateTime", inplace=True)

In [ ]:
meteo_df.loc[meteo_df["Cumulated Rainfall (mm)"] < 0, ["Cumulated Rainfall (mm)"]] = (
    np.nan
)

In [ ]:
# set to nan every value that is equal to -999 or -999.0 in the dataframe
meteo_df.replace(-999, np.nan, inplace=True)

#### Analyze Station

In [ ]:
meteo_info_df = pd.DataFrame(
    index=pd.Index(
        [
            "N Samples",
            "% Missing Values",
            "Frequency (days)",
            "Mean",
            "Std",
            "Start Date",
            "End Date",
        ],
        name="Info",
    ),
    columns=pd.Index(["Parameter"]),
)

#### Timeseries

In [ ]:
for column in meteo_df.columns.difference(diff_columns):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=meteo_df.index,
            y=meteo_df[column],
            mode="lines",
            name=column,
            line=dict(color="blue"),
        )
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=column,
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
    )

    fig.show()

#### Boxplots

In [ ]:
# boxplot of the data
for column in meteo_df.columns.difference(diff_columns):
    fig = go.Figure()
    column_df = meteo_df[column]

    for year in column_df.index.year.unique():
        fig.add_trace(go.Box(y=column_df[column_df.index.year == year], name=year))
    fig.update_layout(
        title=f"{column} at airport",
        xaxis_title="Year",
        yaxis_title=column,
    )

    fig.show()

#### Store Info

In [ ]:
for column in meteo_df.columns.difference(diff_columns):
    start_date = meteo_df[column].dropna().index.min().strftime("%Y-%m-%d")
    end_date = meteo_df[column].dropna().index.max().strftime("%Y-%m-%d")

    df = meteo_df[start_date:end_date][column]

    print(f"Start date for {column}: {start_date}")
    print(f"End date for {column}: {end_date}")

    missing_values = df.isna().sum() / df.shape[0]
    print(f"Missing values for {column}: {missing_values}")

    frequency = df.index.to_series().diff().value_counts().index[0].days
    print(f"Frequency for {column}: {frequency}")

    meteo_info_df.loc["N Samples", column] = meteo_df[column].dropna().shape[0]
    meteo_info_df.loc["% Missing Values", column] = missing_values
    meteo_info_df.loc["Frequency (days)", column] = frequency

    meteo_info_df.loc["Mean", column] = df.mean()
    meteo_info_df.loc["Std", column] = df.std()

    meteo_info_df.loc["Start Date", column] = start_date
    meteo_info_df.loc["End Date", column] = end_date

#### Outliers and Missing Values Imputation

In [ ]:
# define the outliers through the STL decomposition

for column in meteo_df.columns.difference(diff_columns):
    df = meteo_df[column].copy()

    df.dropna(inplace=True)

    df = df.resample("ME").mean()

    df.interpolate(method="time", inplace=True)

    date_range = meteo_df[column].dropna().index
    date_range = meteo_df.min(), date_range.max()

    # make sure that the dataframe starts and finishes in the same month
    start_index = df[df.index.month == date_range[1].month].index[0]

    # Slice the dataframe to start from the found index
    df = df.loc[start_index:]

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df,
            mode="lines",
            name="Original",
        ),
        row=1,
        col=1,
    )

    # ===== Prophet =====

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = forecasting_final["y"] - forecasting_final["yhat"]
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"] - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes" if (np.abs(x["error"]) > factor * x["uncertainty"]) else "No",
        axis=1,
    )

    print("===== Prophet =====")

    # Mean Absolute Error (MAE)
    MAE = mean_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Absolute Error (MAE): " + str(np.round(MAE, 2)))

    # Median Absolute Error (MedAE)
    MEDAE = median_absolute_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Median Absolute Error (MedAE): " + str(np.round(MEDAE, 2)))

    # Mean Squared Error (MSE)
    MSE = mean_squared_error(forecasting_final["yhat"], forecasting_final["y"])
    print("Mean Squared Error (MSE): " + str(np.round(MSE, 2)))

    # Root Mean Squarred Error (RMSE)
    RMSE = np.sqrt(
        int(mean_squared_error(forecasting_final["yhat"], forecasting_final["y"]))
    )
    print("Root Mean Squared Error (RMSE): " + str(np.round(RMSE, 2)))

    # Mean Absolute Percentage Error (MAPE)
    MAPE = mean_absolute_percentage_error(
        forecasting_final["yhat"], forecasting_final["y"]
    )
    print("Mean Absolute Percentage Error (MAPE): " + str(np.round(MAPE, 2)) + " %")

    anomaly = forecasting_final[forecasting_final["anomaly"] == "Yes"]

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["yhat"],
            mode="lines",
            name="Prediction (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["y"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["error"],
            mode="lines",
            name="Error",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecasting_final["ds"],
            y=forecasting_final["uncertainty"],
            mode="lines",
            name="Uncertainty",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=anomaly["ds"],
            y=anomaly["error"],
            mode="markers",
            name="Outliers (Prophet)",
        ),
        row=2,
        col=1,
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        height=800,
        width=1000,
    )

    fig.show()

In [ ]:
# No need to remove outliers
meteo_df.drop(columns=diff_columns, inplace=True)

meteo_df = meteo_df.resample("ME").mean()

meteo_df.interpolate(method="time", inplace=True)

In [ ]:
# Need to compare the air temperature between the airport and the stations first

# plot the air temperature for the airport and the stations
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=meteo_df.index,
        y=meteo_df["Temperature Mean (°C)"],
        mode="lines",
        name="Airport",
        line=dict(color="blue"),
    )
)

for station_id in sw_df["Station"].unique():
    station_df = sw_df[sw_df["Station"] == station_id]

    fig.add_trace(
        go.Scatter(
            x=station_df["DateTime"],
            y=station_df["Air Temperature (°C)"],
            mode="lines",
            name=f"Station {station_id}",
        )
    )

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    font=dict(
        size=18,
    ),
    title="Temperature",
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

In [ ]:
# index to column
meteo_df["DateTime"] = pd.to_datetime(meteo_df.index)

In [ ]:
meteo_df["DateTime"] = meteo_df["DateTime"].astype(int)
scaler = MinMaxScaler()
meteo_df["DateTime"] = scaler.fit_transform(meteo_df[["DateTime"]])

In [ ]:
model = smf.ols(
    f'Q("Temperature Mean (°C)") ~ DateTime',
    data=meteo_df[meteo_df.index >= "1976-08-01"],
).fit()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=meteo_df[meteo_df.index >= "1976-08-01"].index,
        y=meteo_df[meteo_df.index >= "1976-08-01"]["Temperature Mean (°C)"],
        mode="lines",
        name="Air Temperature (°C)",
        line=dict(color="black"),
        opacity=0.5,
    )
)


fig.add_trace(
    go.Scatter(
        x=meteo_df[meteo_df.index >= "1976-08-01"].index,
        y=model.predict(meteo_df[meteo_df.index >= "1976-08-01"]["DateTime"]),
        mode="lines",
        line=dict(color="rgb(200, 2, 110)"),
    )
)

fig.update_layout(
    xaxis_title="Time",
    yaxis_title=column,
    # template='plotly_white',
    showlegend=False,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.6),
    margin=dict(l=10, r=10, t=10, b=10),
)

start_year = meteo_df[meteo_df.index >= "1976-08-01"].index.year.min()
end_year = meteo_df[meteo_df.index >= "1976-08-01"].index.year.max()
tickvals = [
    pd.Timestamp(f"{year}-02-26") for year in range(start_year, end_year + 1, 2)
]
ticktext = [str(year) for year in range(start_year, end_year + 1, 2)]

fig.update_xaxes(
    tickvals=tickvals,
    ticktext=ticktext,
    title_text="Time",
    tickangle=90,  # Add vertical rotation to x-axis labels
)

fig.update_yaxes(
    range=[-10, 30],
)


fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Temperature (°C)",
    font=dict(
        size=18,
    ),
)

fig.show()

In [ ]:
model = smf.ols(
    f'Q("Cumulated Rainfall (mm)") ~ DateTime',
    data=meteo_df[meteo_df.index >= "1976-08-01"],
).fit()

In [ ]:
# do the same with Cumulated Rainfall
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=meteo_df[meteo_df.index >= "1976-08-01"].index,
        y=meteo_df[meteo_df.index >= "1976-08-01"]["Cumulated Rainfall (mm)"],
        mode="lines",
        name="Cumulated Rainfall (mm)",
        line=dict(color="black"),
        opacity=0.5,
    )
)


fig.add_trace(
    go.Scatter(
        x=meteo_df[meteo_df.index >= "1976-08-01"].index,
        y=model.predict(meteo_df[meteo_df.index >= "1976-08-01"]["DateTime"]),
        mode="lines",
        line=dict(color="rgb(200, 2, 110)"),
    )
)

fig.update_layout(
    xaxis_title="Time",
    yaxis_title=column,
    # template='plotly_white',
    showlegend=False,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.6),
    margin=dict(l=10, r=10, t=10, b=10),
)

start_year = meteo_df[meteo_df.index >= "1976-08-01"].index.year.min()
end_year = meteo_df[meteo_df.index >= "1976-08-01"].index.year.max()
tickvals = [
    pd.Timestamp(f"{year}-02-26") for year in range(start_year, end_year + 1, 2)
]
ticktext = [str(year) for year in range(start_year, end_year + 1, 2)]

fig.update_xaxes(
    tickvals=tickvals,
    ticktext=ticktext,
    title_text="Time",
    tickangle=90,  # Add vertical rotation to x-axis labels
)

fig.update_yaxes(
    range=[0, 7],
)


fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Cumulated Rainfall (mm)",
    font=dict(
        size=18,
    ),
)

fig.show()

In [ ]:
# compute pearson correlation

for station_id in sw_df["Station"].unique():
    start_date = sw_df[sw_df["Station"] == station_id]["DateTime"].min()
    end_date = sw_df[sw_df["Station"] == station_id]["DateTime"].max()

    # take the common date range with the airport
    start_date = max(start_date, meteo_df.index.min())
    end_date = min(end_date, meteo_df.index.max())

    airport_df = meteo_df[start_date:end_date].copy()

    # take the common date range with the station
    station_df = sw_df[sw_df["Station"] == station_id]
    station_df = station_df[
        (station_df["DateTime"] >= start_date) & (station_df["DateTime"] <= end_date)
    ]

    # compute pearson correlation
    corr, _ = stats.pearsonr(
        airport_df["Temperature Mean (°C)"],
        station_df["Air Temperature (°C)"],
    )

    rmse = np.sqrt(
        mean_squared_error(
            airport_df["Temperature Mean (°C)"], station_df["Air Temperature (°C)"]
        )
    )
    rmse = rmse / (
        airport_df["Temperature Mean (°C)"].max()
        - airport_df["Temperature Mean (°C)"].min()
    )
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=airport_df.index,
            y=airport_df["Temperature Mean (°C)"],
            mode="lines",
            name="Airport",
            line=dict(color="blue"),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=sw_df[sw_df["Station"] == station_id]["DateTime"],
            y=sw_df[sw_df["Station"] == station_id]["Air Temperature (°C)"],
            mode="lines",
            name=f"Station {station_id}",
        )
    )

    # add the correlation to the plot
    fig.add_annotation(
        x=0.01,
        y=0.95,
        xref="paper",
        yref="paper",
        text=f"Pearson Correlation: {corr:.2f}",
        showarrow=False,
        font=dict(
            size=18,
        ),
    )

    fig.add_annotation(
        x=0.01,
        y=0.90,
        xref="paper",
        yref="paper",
        text=f"RMSE: {rmse:.2f}",
        showarrow=False,
        font=dict(
            size=18,
        ),
    )

    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Air Temperature (°C)",
        font=dict(
            size=18,
        ),
    )

    fig.show()

In [ ]:
# scatter plot of the air temperature between the airport and the stations
for station_id in sw_df["Station"].unique():
    start_date = sw_df[sw_df["Station"] == station_id]["DateTime"].min()
    end_date = sw_df[sw_df["Station"] == station_id]["DateTime"].max()

    # take the common date range with the airport
    start_date = max(start_date, meteo_df.index.min())
    end_date = min(end_date, meteo_df.index.max())

    airport_df = meteo_df[start_date:end_date].copy()

    # take the common date range with the station
    station_df = sw_df[sw_df["Station"] == station_id]
    station_df = station_df[
        (station_df["DateTime"] >= start_date) & (station_df["DateTime"] <= end_date)
    ]

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=airport_df["Temperature Mean (°C)"],
            y=station_df["Air Temperature (°C)"],
            mode="markers",
            name="Data",
            marker=dict(size=8, color="blue", opacity=0.7),
        )
    )

    # add line on bisector
    fig.add_trace(
        go.Scatter(
            x=[-10, 40],
            y=[-10, 40],
            mode="lines",
            name="Bisector",
            line=dict(color="red", width=2, dash="dash"),
        )
    )

    fig.update_layout(
        xaxis_title="Airport",
        yaxis_title=f"Station {station_id}",
        font=dict(
            size=18,
        ),
        title="Air Temperature",
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
    )

    fig.show()

In [ ]:
# reindex the sw_df first to have unique indices
sw_df.reset_index(inplace=True)

In [ ]:
# The correlation is high between the airport and the stations,
# so we can add the airport data variables to the stations

# add the rainfall data to the stations
sw_df["Cumulated Rainfall (mm)"] = np.nan

for station_id in sw_df["Station"].unique():
    start_date = sw_df[sw_df["Station"] == station_id]["DateTime"].min()
    end_date = sw_df[sw_df["Station"] == station_id]["DateTime"].max()

    # take the common date range with the airport
    start_date = max(start_date, meteo_df.index.min())
    end_date = min(end_date, meteo_df.index.max())

    airport_df = meteo_df[start_date:end_date].copy()

    # take the common date range with the station
    # Identify the indices in sw_df that match the station_id and are within the date range
    indices = sw_df[
        (sw_df["Station"] == station_id)
        & (sw_df["DateTime"] >= start_date)
        & (sw_df["DateTime"] <= end_date)
    ].index

    # Directly update sw_df for the matching indices
    sw_df.loc[indices, "Cumulated Rainfall (mm)"] = airport_df[
        "Cumulated Rainfall (mm)"
    ].values

In [ ]:
sw_df["Cumulated Rainfall (mm)"].fillna(value=0, inplace=True)

In [ ]:
# do the same with the temperature, we are going to use the mean temperature of the airport for the stations
for stations_id in sw_df["Station"].unique():
    start_date = sw_df[sw_df["Station"] == station_id]["DateTime"].min()
    end_date = sw_df[sw_df["Station"] == station_id]["DateTime"].max()

    # take the common date range with the airport
    start_date = max(start_date, meteo_df.index.min())
    end_date = min(end_date, meteo_df.index.max())

    airport_df = meteo_df[start_date:end_date].copy()

    # take the common date range with the station
    # Identify the indices in sw_df that match the station_id and are within the date range
    indices = sw_df[
        (sw_df["Station"] == station_id)
        & (sw_df["DateTime"] >= start_date)
        & (sw_df["DateTime"] <= end_date)
    ].index

    # Directly update sw_df for the matching indices
    sw_df.loc[indices, "Air Temperature (°C)"] = airport_df[
        "Temperature Mean (°C)"
    ].values

## Final Cleaning

In [ ]:
sw_df.isna().sum()

In [ ]:
sw_df.drop(columns=["index"], inplace=True)

In [ ]:
# print the date range for every station and variable

for station_id in sw_df["Station"].unique():
    station_df = sw_df[sw_df["Station"] == station_id]

    for column in station_df.columns.difference(["DateTime", "Station"]):
        start_date = station_df[["DateTime", column]].dropna()["DateTime"].min()
        end_date = station_df[["DateTime", column]].dropna()["DateTime"].max()

        print(f"Station {station_id} - {column}")
        print(f"Start date: {start_date}")
        print(f"End date: {end_date}")
        print()

In [ ]:
# Fix ammonium last value
sw_df["Ammonium (mg/l)"].ffill(inplace=True)

In [ ]:
%%script false --no-raise-error

# make it start and finish with the same month
for station_id in sw_df["Station"].unique():
    station_df = sw_df[sw_df["Station"] == station_id]

    for column in station_df.columns.difference(
        ["DateTime", "Station"]
    ):
        df = station_df[["DateTime", column]].copy()

        df.dropna(inplace=True)

        start_index = df[df['DateTime'].dt.month == df['DateTime'].max().month].index[0]
        
        # set to nan until the start index
        df.loc[:start_index, column] = np.nan
        
        station_df.loc[df.index, column] = df[column]
        
        # update the sw_df
        sw_df.loc[station_df.index, column] = station_df[column]

In [ ]:
%%script false --no-raise-error
sw_df.dropna(
    subset=station_df.columns.difference(["DateTime", "Station"]), how="all", inplace=True
)

In [ ]:
sw_df

In [ ]:
sw_df.to_excel(os.path.join(clean_data_folder, "berlin.xlsx"), index=False)